# Assignment 1
Linear and circular convolution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy
from scipy.linalg import toeplitz, circulant, dft

plt.rcParams.update({
    "font.serif":["cm"],
    "font.size": 12})

colors = {
    "blue": "#377eb8",
    "orange": "#ff7f00",
    "green": "#4daf4a",
    "pink": "#f781bf",
    "brown": "#a65628",
    "purple": "#984ea3",
    "gray": "#999999",
    "red": "#e41a1c",
    "yellow": "#dede00",
    "black": "#000000",
}

## Part 1: Starter Exercise

Before using built-in functions, write the convolution function once!

**The Goal:** Implement 1D Linear Convolution (`mode='full'`) in three ways and verify they are identical.
1. **Manual Loops:** To appreciate the computational complexity.
2. **Matrix Algebra:** To leverage linear algebra tools.
3. **Built-in:** To make things more efficient.

In [ ]:
def manual_convolution_loops(x, h, mode='full'):
    """
    Computes convolution y[n] = sum(x[k] * h[n-k]) using nested loops.
    Supports 'full', 'same', and 'valid' modes.
    TODO: Implement the convolution logic here
    """
    # TODO: Initialize output array
    # TODO: Loop over output samples
    # TODO: Handle different modes (full, same, valid)
    pass
    
def test_implementations():
    x = np.array([1, 2, 3, 4, 5])
    h = np.array([1, -1, 2])
    
    print(f"Input x: {x}")
    print(f"Filter h: {h}")
    print(f"-"*30)

    for mode in ['full', 'same', 'valid']:
        # 1. Manual Implementation
        y_manual = manual_convolution_loops(x, h, mode=mode)
        
        # 2. Built-in Implementation (Numpy)
        y_builtin = np.convolve(x, h, mode=mode)
        
        print(f"Mode '{mode}':\")
        print(f"  Manual:   {y_manual}")
        print(f"  Built-in: {y_builtin}")
        
        if np.allclose(y_manual, y_builtin):
            print(f"  Result:   MATCH")
        else:
            print(f"  Result:   MISMATCH")
    print(f"-"*30)

test_implementations()

## Part 2: Linear Convolution as a Toeplitz Matrix

Now that we understand the loops, we show that convolution is simply a matrix-vector product $y = Ax$.

In [ ]:
def demo_toeplitz_convolution():
    # Define a simple signal x and filter h
    x = np.array([1, 2, 3, 4])
    h = np.array([1, -1, 2]) # Length 3
    
    # Standard Numpy Convolve
    y_numpy = np.convolve(x, h, mode='full')
    print(f"Standard Convolution Output: {y_numpy}")
    
    # TODO: Construct Toeplitz matrix
    # H_mat = ...
    
    print("\nToeplitz Convolution Matrix (H):")
    # print(H_mat)
    
    # TODO: Perform matrix multiplication
    # y_matrix = ...
    # print(f"\nMatrix Multiplication Output: {y_matrix}")
    
    # assert np.allclose(y_numpy, y_matrix), "Mismatch between numpy and matrix method!"

demo_toeplitz_convolution()

def demo_circulant_matrix():
    x = np.array([1, 2, 3, 4])
    h = np.array([1, 0, -1, 0]) # Same length as x for circular conv
    
    # TODO: Create circulant matrix from h
    # C = ...
    # print("Circulant Matrix C:")
    # print(C)
    
    # TODO: Compare matrix multiplication with FFT-based circular convolution
    # y_matrix = C @ x
    # y_fft = np.fft.ifft(np.fft.fft(x) * np.fft.fft(h)).real
    # print(f"\nMatrix Result: {y_matrix}")
    # print(f"FFT Result: {y_fft}")
    
demo_circulant_matrix()

## Part 3: Audio De-reverberation (1D Application)

Use the functions to construct reverberated audio and perform de-reverberation
* **Model:** Room impulse response
* **Input:** Standard speech file
* **Restoration:** Inverse/Wiener filter

In [ ]:
import scipy.io.wavfile as wavfile
import urllib.request
import io

In [ ]:
def get_audio_sample():
    """Downloads a standard wav file or generates a fallback."""
    url = "https://www.signalogic.com/melp/EngSamples/Orig/male.wav"
    try:
        print(f"Attempting to download audio from {url}...")
        response = urllib.request.urlopen(url)
        data = response.read()
        sr, y = wavfile.read(io.BytesIO(data))
        # Normalize to -1..1
        y = y.astype(float) / np.max(np.abs(y))
        # Take a 3-second slice to keep processing fast
        y = y[:sr*3]
        print("Download successful.")
        return sr, y
    except Exception as e:
        print(f"Download failed: {e}. Generating fallback audio.")

In [ ]:
def create_room_impulse_response(sr):
    """Creates a synthetic Room Impulse Response (Echo)."""
    # A decaying exponential with random spikes
    T_echo = 0.3 # 300ms reverb
    n_echo = int(sr * T_echo)
    t = np.linspace(0, T_echo, n_echo)
    
    # Exponential decay envelope
    envelope = np.exp(-10 * t)
    
    # Random reflections
    noise = np.random.randn(n_echo)
    rir = noise * envelope
    
    # Normalize energy
    rir = rir / np.sqrt(np.sum(rir**2))
    return rir

In [ ]:
def wiener_deconvolution(observed, kernel, K):
    """
    Apply Wiener Deconvolution in Frequency Domain.
    TODO: Implement H_hat = H* / (|H|^2 + K)
    """
    pass

In [ ]:
# 1. Setup
sr, clean_speech = get_audio_sample()
rir = create_room_impulse_response(sr)

# 2. Simulate "The Room" (Convolution)
reverbed_speech = np.convolve(clean_speech, rir, mode='same')

# 3. Add Noise (The "Hiss")
noise_level = 0.02
noise = np.random.randn(len(reverbed_speech)) * noise_level
observed_signal = reverbed_speech + noise

# 4. Attempt Recovery
# recovered_naive = ...
# recovered_wiener = ...

In [ ]:
# 5. Visualization
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.title("Clean Speech (Source)")
plt.plot(clean_speech, color=colors['blue'])
plt.grid(True)

plt.subplot(4, 1, 2)
plt.title("Reverberant + Noisy (Measurements)")
plt.plot(observed_signal, color=colors['orange'])
plt.grid(True)

plt.subplot(4, 1, 3)
plt.title("Naive Inverse Filter")
plt.plot(recovered_naive, color=colors['red'])
plt.grid(True)

plt.subplot(4, 1, 4)
plt.title("Wiener Filter Restoration")
plt.plot(recovered_wiener, color=colors['green'])
plt.grid(True)
plt.xlabel("$n$")

plt.tight_layout()
plt.show()

In [ ]:
import IPython.display as ipd

print("1. Clean Speech")
ipd.display(ipd.Audio(clean_speech, rate=sr))

print("2. Reverberant + Noisy")
ipd.display(ipd.Audio(observed_signal, rate=sr))

print("3. Naive Inverse Filter")
ipd.display(ipd.Audio(recovered_naive, rate=sr))

print("4. Wiener Restored")
ipd.display(ipd.Audio(recovered_wiener, rate=sr))

## Part 4/5: 2D Image Deblurring (Set5 & Levin Kernels)

We apply the same Wiener Filter logic to images.
* **Gaussian Blur:** Simulates out-of-focus lens.
* **Motion Blur:** Simulates camera shake (Levin-like kernels).
* **Inverse Filter:** Fails due to zeros in the blur kernel.
* **Wiener Filter:** Restores structure by regularizing the inversion.

In [ ]:
from PIL import Image
def get_image_sample():
    """Downloads 'baby.png' from Set5."""
    url = "https://raw.githubusercontent.com/jbhuang0604/SelfExSR/master/data/Set5/image_SRF_4/img_001_SRF_4_HR.png"
    try:
        print(f"Downloading Set5 image from {url}...")
        with urllib.request.urlopen(url) as response:
            data = response.read()
        img = Image.open(io.BytesIO(data)).convert('L') # Convert to Grayscale
        img = np.array(img).astype(float) / 255.0
        print("Download successful.")
        return img
    except Exception as e:
        print(f"Download failed ({e}). Generating synthetic text image...")

In [ ]:
def get_gaussian_kernel(size=15, sigma=3):
    """Generates a 2D Gaussian Kernel."""
    # TODO: Implement Gaussian kernel generation
    pass

def get_motion_kernel(size=15, angle=45):
    """Generates a Motion Blur Kernel (Line)."""
    kernel = np.zeros((size, size))
    # Draw a line in the center using a simple algorithm
    center = size // 2
    slope = np.tan(np.deg2rad(angle))
    for x in range(size):
        offset = int((x - center) * slope)
        y = center - offset
        if 0 <= y < size:
            kernel[y, x] = 1.0
    return kernel / np.sum(kernel)

In [ ]:
def wiener_deconvolution_2d(img_blurred, kernel, K):
    """
    Performs 2D Wiener Deconvolution in Frequency Domain.
    """
    # TODO: Implement Wiener Deconvolution
    # TODO: Pad kernel to image size
    # TODO: Wiener filter in the frequency domain
    pass

In [ ]:
# 1. Setup Data
img = get_image_sample()
k_motion = get_motion_kernel(size=15, angle=30)

# 2. Simulate Degradation
# We use fftconvolve for accurate boundary handling (or standard convolve2d)
img_blurred = scipy.signal.convolve2d(img, k_motion, mode='same', boundary='wrap')

# 3. Add Noise
noise_sigma = 0.01  # Gaussian noise
noise = np.random.randn(*img.shape) * noise_sigma
img_degraded = img_blurred + noise

# 4. Restoration
# K should be approx (Noise Power / Signal Power) or tuned manually
# Try K=0 for Inverse Filter (will explode)
img_inverse = wiener_deconvolution_2d(img_degraded, k_motion, K=1e-12) 
# Try Tuned K for Wiener
K = 1e-2
img_wiener = wiener_deconvolution_2d(img_degraded, k_motion, K=K)

In [ ]:
# 5. Visualization
plt.figure(figsize=(8, 8))

plt.subplot(2, 2, 1)
plt.title("Original Image (Set5)")
plt.imshow(img, cmap='gray')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.title(f"Degraded ({blur_type} + Noise)")
plt.imshow(img_degraded, cmap='gray')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.title("Inverse Filter (Unregularized)\nNote: Noise Amplification Pattern")
plt.imshow(np.clip(img_inverse, 0, 1), cmap='gray')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.title(f"Wiener Filter (K={K})\nRestored Structure")
plt.imshow(np.clip(img_wiener, 0, 1), cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()

## Part 6: Blind Deconvolution

- **Observation:** Motion blur, "unknown"
- **Restoration:** Parametrize the kernel as Gaussian and find parameters

In [ ]:
def get_mystery_image():
    """Loads the 'Page' image."""
    try:
        print(f"Loading mystery image from local file...")
        img = Image.open("data/page.png").convert('L')
        img = np.array(img).astype(float) / 255.0
        return img
    except Exception as e:
        print(f"Failed to load.")

In [ ]:
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return 100
    return 10 * np.log10(1.0 / mse)

# 1. Prepare the 
gt_img = get_mystery_image()
# mystery_sigma = 2.4
# mystery_kernel = get_gaussian_kernel(size=21, sigma=mystery_sigma)
mystery_kernel = get_motion_kernel(size=7, angle=30)
mystery_blur = scipy.signal.convolve2d(gt_img, mystery_kernel, mode='same', boundary='wrap')
mystery_noise = np.random.randn(*gt_img.shape) * 0.02
mystery_observed = mystery_blur + mystery_noise

In [ ]:
# TODO: Restore using an estimate of the kernel as Gaussian filter
# TODO: Find the optimal parameters
# TODO: Display results and PSNR